This notebook allows to compute different clustering algorithms on the pmlb datasets

All path configuration and number of processor to use are in `config.py`

In [1]:
import config
from utils.utils import print_verbose

Ran on 22 April 2020 13:30:10
Local


# Import

In [2]:
# Configuration computation
from utils.fold import createFold, readFold, computedFold

In [3]:
# Datasets import
from pmlb import classification_dataset_names, fetch_data

In [4]:
# Constraint methods
from scipy.sparse import find
from utils.constraint import random_indices, generate_constraint, completion_constraint

In [5]:
# Normalization
from sklearn.preprocessing import StandardScaler

In [6]:
# Kernels methods
from kernels.features import produce_kernels, normalize_and_check_kernels
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics.pairwise import euclidean_distances

In [7]:
# Model imports
## R Model of constrained clustering
## Require to have R and have installed conclust library
from rpy2.robjects.packages import importr
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()
clusterR = importr('conclust')

from sklearn.cluster import KMeans
from utils.clustering import Clustering

## Metrics Learning
from metric_learn import MMC, ITML

## HMRF
from models.hmrf import hmrf_kmeans

## Cosine bayesian optimization
from models.cosine_bayes_opt import cosine_bayes_clustering

## Mahalanobius bayesian optimization
from models.mahalanobis import mahalanobis_bayes_clustering, mahalanobis_tpe_clustering

## SSGCA
from SemiSupervisedGraphClustering.sskkmeans import holdOutSskmeans

## Cross validation
from models.cross_validation import crossValidation

## Our model
from models.kernel_opt import kernel_clustering

In [8]:
# Evaluation methods
from utils.metrics import evalSplit

In [9]:
# For reproductibility
import numpy as np
np.random.seed(0)

In [10]:
# Save
import pickle
import os

In [11]:
# Creates folder for save
os.mkdir(os.path.join(config.result, config.time))

In [12]:
## Force recomputation kernels and configuration
force = False

# Configuration Computation

In order to compute separetly any method, we save the configuration that we want to apply for all methods

In [13]:
if force:
    for dname in sorted(classification_dataset_names):
        createFold(dname, verbose = 2)

# Models computation

Second we define a function which opens the data, computes the constraints and then computes the different algorithms and saves the results.  

In [14]:
testMode = False

# Kernels
## Kernel to compute
kernels_name = ['rbf', 'sigmoid', 'polynomial', 
                'laplacian', 'linear']

## Kernels options -- Refer to kernels/features.py
### Used only with Bayesian Optimization method
kernel_args = {"normalize": "expectation", 
               "check_method": "trivial", 
               "clip": True}

In [15]:
def compute(dname, algorithm, percentageConstraint = (True, 10), verbose = 0, n_jobs = 1, folder = None, normalize = True):
    """
        Computes the given algorithm(s) and
        Saves the performances on the dataset dname
        
        percentageConstraint for the constraint matrix (between 0 and 100)
    """
    if folder is None:
        folder = os.path.join(config.result, config.time)
    save_file = os.path.join(folder, dname + "_{}.pickle".format(percentageConstraint))
    
    # Verify if the file has already been computed
    if os.path.isfile(save_file):
        print_verbose("File {} found".format(save_file), verbose)
        return pickle.load(open(save_file, 'rb'))
    
    # Read configuration
    configuration = readFold(dname)
    if configuration is None:
        print_verbose("No configuration found for {}".format(dname), verbose)
        return None
    
    # Read data and put them in good format for sklearn
    data, labelvector = fetch_data(dname, return_X_y = True, local_cache_dir = config.datadir)
    data = data.astype('float64')
    classes = configuration["N_Classes"]
    
    if len(data) > 50000:
        # Limit number of points to evaluate
        return None
    
    if "CSC" in algorithm:
        if 'Approximation' in algorithm and len(data) < 2000:
            # Limit number of points to evaluate
            return None
        elif 'Approximation' not in algorithm and len(data) > 2000:
            return None
    
    print_verbose("{} - {} points divided in {} classes".format(dname, len(data), classes), verbose)
    
    # Computes the kernels for the given data
    if  "Cross Validation" == algorithm or "CSC" in algorithm:
        print_verbose("Computation Kernels", verbose)
        names, kernels = produce_kernels(dname, kernels_name, data, force = force, verbose = verbose, n_jobs = n_jobs, 
                                         approximation = "Approximation CSC" == algorithm)

        print_verbose("Normalization Kernels", verbose)
        args = {"normalize": "approximation"} if "Approximation CSC" == algorithm else kernel_args
        names, kernels = normalize_and_check_kernels(names, kernels, classes, verbose = verbose, n_jobs = n_jobs, **args)
    
        ## If no kernel => Change computation 
        if len(kernels) == 0:
            print_verbose("Kernels Default", verbose)
            return None
        
    # Normalization
    if normalize:
        print_verbose("Normalize", verbose)
        scaler = StandardScaler()
        data = scaler.fit_transform(data)

    # Affinity for SSGCA
    if "SSK-Kmeans" == algorithm:
        _, affinities = produce_kernels(dname, ['rbf'], data, verbose = verbose, n_jobs = n_jobs)

    # Iteration for confidence
    scoreIt, assignationIt, constraintIt = {}, {}, {}
    for fold in configuration["Train"]:
        print_verbose("Iteration {} / {}".format(fold + 1, len(configuration["Train"])), verbose)

        # Read precomputed indices for train and constraint
        train, constraint_indices = configuration["Train"][fold], configuration["Constraint"][fold]
        
        # Compute constraints matrix
        ## Number constraint
        per, number_constraint = percentageConstraint
        if per:
            number_constraint = int((number_constraint*(len(train)-1)*len(train)/2.)/100.)
        elif number_constraint > len(constraint_indices):
            print_verbose("Too many constraint for the dataset", verbose)
            return None
        
        ## Enforce max number constraints
        number_constraint = min(number_constraint, 1000)

        ## Subselect the constraint matrix
        constraint = generate_constraint(labelvector, constraint_indices[:number_constraint])
    
        ### Allows a subset to be evaluated
        if "SSK-Kmeans" == algorithm:
            hold = int(number_constraint * 0.9)
            hold = generate_constraint(labelvector, constraint_indices[:hold])

        ## Completion Constraint Matrix
        print_verbose("Completion Constraint", verbose)
        if constraint.shape[0] < 5000:
            constraint = completion_constraint(constraint)
            if "SSK-Kmeans" == algorithm:
                hold = completion_constraint(hold)

        ## Stop if no constraint
        if len(find(constraint)) == 0:
            print_verbose("No constraint", verbose)
            continue
            
        ## R Format constraints and metric learning
        must_link = [((i, j), (j, i)) for i, j, v in zip(*find(constraint)) if v > 0]
        cannot_link = [((i, j), (j, i)) for i, j, v in zip(*find(constraint)) if v < 0]
        must_link, cannot_link = np.vstack(must_link), np.vstack(cannot_link)
        const_metric = must_link[:, 0], must_link[:, 1], cannot_link[:, 0], cannot_link[:, 1]
        must_link, cannot_link = must_link + 1, cannot_link + 1 # +1 for R index

        # Computes model(s)
        assignation, score = {}, {}
        
        try:
            print_verbose(algorithm, verbose)
            ## R Models
            if "COP K-means" == algorithm:
                for cop_iteration in range(5):
                    assignation["COP K-means"] = np.array(clusterR.ckmeans(data, classes, must_link, cannot_link))
                    if len(np.unique(assignation["COP K-means"])) == classes:
                        break
                score["COP K-means"] = evalSplit(assignation["COP K-means"], labelvector, train)

            if "lcvqe" == algorithm:
                assignation["lcvqe"] = np.array(clusterR.lcvqe(data, classes, must_link, cannot_link))
                score["lcvqe"] = evalSplit(assignation["lcvqe"], labelvector, train)

            if "mpckm" == algorithm:
                assignation["mpckm"] = np.array(clusterR.mpckm(data, classes, must_link, cannot_link))
                score["mpckm"] = evalSplit(assignation["mpckm"], labelvector, train)

            if "kmeans" == algorithm:
                assignation["kmeans"] = KMeans(classes).fit(data).labels_
                score["kmeans"] = evalSplit(assignation["kmeans"], labelvector, train)
                
            
            ## HMRF
            if "hmrf" == algorithm:
                assignation["hmrf"] = hmrf_kmeans(data, classes, constraint)
                score["hmrf"] = evalSplit(assignation["hmrf"], labelvector, train)
            
            
            ## Cosine Learning
            if "Bayesian cosine" == algorithm:
                clustering = Clustering.create("kernelKmeans", classes = classes, constraint_matrix = constraint)
                assignation["Bayesian cosine"] = cosine_bayes_clustering(data, clustering, constraint, verbose = verbose)
                score["Bayesian cosine"] = evalSplit(assignation["Bayesian cosine"], labelvector, train)
                
            ## Mahalanobis 
            if "Bayesian mahalanobis" == algorithm:
                clustering = Clustering.create("kmeans", classes = classes, constraint_matrix = constraint)
                assignation["Bayesian mahalanobis"] = mahalanobis_bayes_clustering(data, clustering, constraint, verbose = verbose)
                score["Bayesian mahalanobis"] = evalSplit(assignation["Bayesian mahalanobis"], labelvector, train)

            if "TPE mahalanobis" == algorithm:
                clustering = Clustering.create("kmeans", classes = classes, constraint_matrix = constraint)
                assignation["TPE mahalanobis"] = mahalanobis_tpe_clustering(data, clustering, constraint, verbose = verbose)
                score["TPE mahalanobis"] = evalSplit(assignation["TPE mahalanobis"], labelvector, train)
                
            ## Distance Learning and LCVQE
            if "mmc" == algorithm:
                transformed = MMC().fit_transform(data, const_metric)
                assignation["mmc"] = np.array(clusterR.lcvqe(transformed, classes, must_link, cannot_link))
                score["mmc"] = evalSplit(assignation["mmc"], labelvector, train)
            
            if "itml" == algorithm:
                transformed = ITML().fit_transform(data, const_metric)
                assignation["itml"] = np.array(clusterR.lcvqe(transformed, classes, must_link, cannot_link))
                score["itml"] = evalSplit(assignation["itml"], labelvector, train)
            
            
            ## SSKmeans
            if "SSK-Kmeans" == algorithm:
                assignation["SSK-Kmeans"] = holdOutSskmeans(affinities, classes, "ratio cut", hold * len(data) / (classes * hold.count_nonzero()), evaluation = constraint * len(data) / (classes * constraint.count_nonzero()))
                score["SSK-Kmeans"] = evalSplit(assignation["SSK-Kmeans"], labelvector, train)
            
            
            ## Simple cross validation
            if "Cross Validation" == algorithm:
                clustering = Clustering.create("kernelKmeans", classes = classes, constraint_matrix = constraint)
                assignation["Cross Validation"], assignation["Cross Validation SoftKmeans"] = crossValidation(kernels, clustering, constraint)
                score["Cross Validation"] = evalSplit(assignation["Cross Validation"], labelvector, train)
                score["Cross Validation SoftKmeans"] = evalSplit(assignation["Cross Validation SoftKmeans"], labelvector, train)  
               
            if "Approximation CSC" == algorithm:
                clustering = Clustering.create("kmeans", classes = classes, constraint_matrix = constraint)
                assignation[algorithm] = kernel_clustering(kernels, clustering, constraint, verbose = verbose, kernel_approx = "Approximation" in algorithm)
                score[algorithm] = evalSplit(assignation[algorithm], labelvector, train)
            
            if "CSC" == algorithm:
                clustering = Clustering.create("kernelKmeans", classes = classes, constraint_matrix = constraint)
                assignation[algorithm] = kernel_clustering(kernels, clustering, constraint, verbose = verbose, kernel_approx = "Approximation" in algorithm)
                score[algorithm] = evalSplit(assignation[algorithm], labelvector, train)

            # Add results
            constraintIt[fold] = np.mean(np.abs(constraint))
            scoreIt[fold], assignationIt[fold] = score, assignation
                    
        except Exception as e:
            raise(e)
            print_verbose("No clustering respecting the constraints", verbose)
            continue

    # Save results
    info = {"Name": dname, "Score": scoreIt, "Assignation": assignationIt, "Percentage Constraint": constraintIt}
    pickle.dump(info, open(save_file, 'wb'))
    
    return info

# Computation

In [19]:
algorithms = ["CSC", "Approximation CSC"]

In [17]:
for algo in algorithms:
    folder = os.path.join(config.result, config.time, algo) # Run method
    os.mkdir(folder)

In [ ]:
if True:
    for dname in computedFold():
        for algo in algorithms:
            compute(dname, algo, (True, 100), verbose = 2, n_jobs = config.processor, folder = os.path.join(config.result, config.time, algo))
else:
    from multiprocessing import Pool
    with Pool(config.processor) as pool:
        pool.starmap(compute, [(dname, algo, (True, 10), 0, 1, os.path.join(config.result, config.time, algo)) for algo in algorithms for dname in computedFold()[::-1]])

cloud - 108 points divided in 4 classes
Computation Kernels
Computing kernels on raw
File /home/vincent/Bureau/CMU/Project/ConstrainedClustering/kernel_files/cloud/raw.npz used
Computing kernels on scaled
File /home/vincent/Bureau/CMU/Project/ConstrainedClustering/kernel_files/cloud/scaled.npz used
Normalization Kernels
Normalization method : expectation
Verification kernel method : <function is_trivial at 0x7f6b6cba5e60>
NOT added: rawrbf - gamma 5.551392242681559e-05
Added: rawrbf - gamma 0.00013878480606703897
Added: rawrbf - gamma 0.00027756961213407794
Added: rawrbf - gamma 0.00041635441820111693
Added: rawrbf - gamma 0.0005551392242681559
NOT added: rawsigmoid - coef0 0.1 gamma 1.2345601966644519e-07
NOT added: rawsigmoid - coef0 0.1 gamma 1.2345601966644519e-06
NOT added: rawsigmoid - coef0 1 gamma 1.2345601966644519e-07
NOT added: rawsigmoid - coef0 1 gamma 1.2345601966644519e-06
NOT added: rawpolynomial - degree 2 coef0 0.1 gamma 1.2345601966644518e-05
Added: rawpolynomial - d

Step 313 - KTA 0.65242
Step 314 - KTA 0.64957
Step 315 - KTA 0.64103
Step 316 - KTA 0.64103
Step 317 - KTA 0.63818
Step 318 - KTA 0.61538
Step 319 - KTA 0.66952
Step 320 - KTA 0.63818
Step 321 - KTA 0.65527
Step 322 - KTA 0.63818
Step 323 - KTA 0.62393
Step 324 - KTA 0.64672
Step 325 - KTA 0.66382
Step 326 - KTA 0.62963
Step 327 - KTA 0.59829
Step 328 - KTA 0.64387
Step 329 - KTA 0.63533
Step 330 - KTA 0.66382
Step 331 - KTA 0.63818
Step 332 - KTA 0.64103
Step 333 - KTA 0.63818
Step 334 - KTA 0.65812
Step 335 - KTA 0.61823
Step 336 - KTA 0.63533
Step 337 - KTA 0.64957
Step 338 - KTA 0.65527
Step 339 - KTA 0.59829
Step 340 - KTA 0.63818
Step 341 - KTA 0.62678
Step 342 - KTA 0.62393
Step 343 - KTA 0.61823
Step 344 - KTA 0.63818
Step 345 - KTA 0.64672
Step 346 - KTA 0.66097
Step 347 - KTA 0.62963
Step 348 - KTA 0.63533
Step 349 - KTA 0.64387
Step 350 - KTA 0.63818
Step 351 - KTA 0.63818
Step 352 - KTA 0.62678
Step 353 - KTA 0.65812
Step 354 - KTA 0.64672
Step 355 - KTA 0.66097
Step 356 - 

------

# Evolution 

How our model behave with less constraints ? We evaluate the performances of different model at different number of constraint

In [ ]:
# Percentage of constraints to explore
percentages = [0.1, 0.2, 0.3, 0.4, 0.5, 1, 2, 3, 4, 5, 10, 20, 30, 40, 50, 75, 100]

In [ ]:
# Run method
if testMode:
    for dname in computedFold():
        for algo in algorithms:
            for percentage in percentages:
                compute(dname, [algo], percentageConstraint = (True, percentage), verbose = 2, n_jobs = config.processor, folder = os.path.join(config.result, config.time, algo))
else:
    from multiprocessing import Pool
    with Pool(config.processor) as pool:
        pool.starmap(compute, [(dname, [algo], (True, percentage), os.path.join(config.result, config.time, algo)) for number in numbers for algo in algorithms for dname in computedFold()])

In [ ]:
numbers = np.arange(50, 1000, 50)

In [ ]:
# Run method
if testMode:
    for dname in computedFold():
        for algo in algorithms:
            for number in numbers:
                compute(dname, [algo], percentageConstraint = (False, number), verbose = 2, n_jobs = config.processor, folder = os.path.join(config.result, config.time, algo))
else:
    from multiprocessing import Pool
    with Pool(config.processor) as pool:
        pool.starmap(compute, [(dname, [algo], (False, number), 0, 1, os.path.join(config.result, config.time, algo)) for number in numbers for algo in algorithms for dname in computedFold()])